In [3]:
Pkg.add("Distributions")
using Distributions

INFO: Nothing to be done
INFO: Precompiling module Distributions.


In [9]:
convolution(z,p)=sum([cdf(Binomial(20,p),x-z)*pdf(Binomial(20,p),x) for x in 1:20])

convolution (generic function with 1 method)

In [12]:
function errorrates(h0, h1,c)
    findfirst(convolution(floor(Integer,,h0)

20-element Array{Float64,1}:
 0.433405   
 0.307373   
 0.200612   
 0.119708   
 0.0649201  
 0.03182    
 0.014018   
 0.00551904 
 0.00192995 
 0.000595285
 0.000160654
 3.75703e-5 
 7.52312e-6 
 1.27044e-6 
 1.77349e-7 
 1.99098e-8 
 1.72677e-9 
 1.08543e-10
 4.39866e-12
 8.62481e-14

In [13]:
c_star(c) = 4*log(4/3)*log(c)/(log(3/2*2))

c_star (generic function with 1 method)